# 📦 Project: The Shipping Crisis (Saudi Retail Case Study)
## 📋 The Scenario
You have been hired by a major Saudi-based e-commerce platform. Management has noticed a significant decline in customer satisfaction scores. Initial reports suggest that a massive number of packages are not reaching customers on time.

**The Management's Question:**
*"We are losing money and customers. We need to know: Is this a logistics problem (Mode of Shipment), a product problem (Weight/Cost), or a strategy problem (Discounts/Priority)? Find the root cause and build a system to save our customers."*

## Phase 1: Data Understanding
In this phase, explore the `Saudi_Retail_Logistics_Data.csv` to find out **why** delays are happening.

* **Step 1.1:** Load the dataset.
* **Step 1.2:** Perform statistical profiling (`.describe()`, `.info()`).
* **Step 1.3:** Create visualizations (Histograms/Scatter) to find the problem.

| Feature Name         | Type       | Description                                                                 | Values / Examples                               |
| :------------------- | :--------- | :-------------------------------------------------------------------------- | :--------------------------------------------- |
| Tracking_ID          | String     | A unique, encrypted alphanumeric identifier for each shipment.              | E3B0C44298FC, 1C2D3E4F5G6H                     |
| Warehouse_Zone       | Categorical| The regional distribution hub handling the order.                           | North_Hub, South_Hub, East_Hub, West_Hub, Central_Port |
| Mode_of_Shipment     | Categorical| The logistics method used to transport the product.                         | Premium_Air, Standard_Sea, Ground_Freight      |
| Customer_Inquiries   | Integer    | Number of times the customer contacted support regarding this order.        | 2, 3, 4, etc.                                  |
| Customer_Score       | Integer    | The rating provided by the customer to the store before shipment (1 is lowest, 5 is highest).            | 1, 2, 3, 4, 5                                  |
| Product_Cost_SAR     | Float      | The value of the item in Saudi Riyals (SAR).                                | 450.00, 1125.50, etc.                          |
| Previous_Orders      | Integer    | Number of successful orders this customer has placed previously.            | 0, 1, 2, 3, etc.                               |
| Product_Priority     | Categorical| The priority level assigned to the product for dispatch.                    | low, medium, high                              |
| Gender               | Categorical| The gender of the customer.                                                 | F, M                                           |
| Discount_offered     | Integer    | The discount amount applied to this specific order.                         | 1, 10, 65, etc.                                |
| Product_Weight_g     | Integer    | The weight of the package in grams.                                         | 1200, 4500, etc.                               |
| is_delayed           | Binary     | Target Variable: Indicates if the shipment arrived past the deadline.       | 1 (Delayed), 0 (On Time)                       |

### **Step 1.1:** Load the dataset.


In [34]:
from pathlib import Path
import pandas as pd
local_file = Path('Saudi_Retail_Logistics_Data.csv')

if not local_file.exists():
    print("Downloading data...")
    df = pd.read_csv('https://drive.google.com/uc?export=download&id=1vayY_XKPwe5lOaq0VNia22O4DkDVXmzv')
    df.to_csv(local_file, index=False)
else:
    print("Loading data from cache...")
    df = pd.read_csv(local_file)
df.head()

Loading data from cache...


,Tracking_ID,Warehouse_Zone,Mode_of_Shipment,Customer_Inquiries,Customer_Score,Product_Cost_SAR,Previous_Orders,Product_Priority,Gender,Discount_offered,Product_Weight_g,is_delayed
0,D66B138C54AB,West_Hub,Ground_Freight,4,1,978.75,2,medium,F,9,4158,0
1,8FF9538E65E6,Central_Port,Standard_Sea,3,3,607.50,2,low,F,26,1659,1
2,D098284FA376,South_Hub,Standard_Sea,3,2,828.75,2,medium,M,9,4466,0
3,9202E9FDFF40,West_Hub,Standard_Sea,3,5,588.75,4,medium,F,2,4640,0
4,16B30490A644,North_Hub,Premium_Air,2,5,1020.00,2,low,M,24,3638,1


### **Step 1.2:** Perform statistical profiling (`.describe()`, `.info()`).


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8799 entries, 0 to 8798
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Tracking_ID         8799 non-null   object 
 1   Warehouse_Zone      8799 non-null   object 
 2   Mode_of_Shipment    8799 non-null   object 
 3   Customer_Inquiries  8799 non-null   int64  
 4   Customer_Score      8799 non-null   int64  
 5   Product_Cost_SAR    8799 non-null   float64
 6   Previous_Orders     8799 non-null   int64  
 7   Product_Priority    8799 non-null   object 
 8   Gender              8799 non-null   object 
 9   Discount_offered    8799 non-null   int64  
 10  Product_Weight_g    8799 non-null   int64  
 11  is_delayed          8799 non-null   int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 825.0+ KB


This output shows that the dataset contains 8,799 rows and 12 columns with no missing values, indicating good data quality. It also confirms the correct data types, with a mix of categorical and numerical variables, and a binary target variable.

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Customer_Inquiries,8799.0,4.049096,1.138649,2.0,3.0,4.0,5.00,7.0
Customer_Score,8799.0,2.997386,1.411194,1.0,2.0,3.0,4.00,5.0
Product_Cost_SAR,8799.0,788.363877,180.192439,360.0,637.5,802.5,941.25,1162.5
Previous_Orders,8799.0,3.581998,1.532722,2.0,3.0,3.0,4.00,10.0
Discount_offered,8799.0,13.366974,16.147015,1.0,4.0,7.0,10.00,65.0
Product_Weight_g,8799.0,3631.227867,1634.180615,1001.0,1838.0,4140.0,5045.00,7846.0
is_delayed,8799.0,0.596659,0.490596,0.0,0.0,1.0,1.00,1.0


This summary provides the statistical distribution of numerical features, including mean, spread, and range. It reveals that about 60% of shipments are delayed, product weights and costs vary widely, discounts are highly skewed, and customer scores are centered around an average rating of 3.

### **Step 1.3:** Create visualizations (Histograms/Scatter) to find the problem.

In [24]:
import plotly.express as px
import pandas as pd

### Delay Distribution

Evaluates whether shipment delays are isolated incidents or a systemic problem. High delay rates indicate a crisis.

In [7]:
df["is_delayed"].value_counts(normalize=True)

is_delayed
1    0.596659
0    0.403341
Name: proportion, dtype: float64

In [ ]:
df_viz = df.copy()

mappings = {
    "Gender": {
        "M": "Male",
        "F": "Female"
    },
    "is_delayed": {
        0: "On Time",
        1: "Delayed"
    },
    "Mode_of_Shipment": {
        "Ground_Freight": "Ground Freight",
        "Standard_Sea": "Standard Sea",
        "Premium_Air": "Premium Air"
    },
    "Product_Priority": {
        "low": "Low Priority",
        "medium": "Medium Priority",
        "high": "High Priority"
    },
    "Warehouse_Zone": {
        "Central_Port": "Central Port",
        "North_Hub": "North Hub",
        "South_Hub": "South Hub",
        "East_Hub": "East Hub",
        "West_Hub": "West Hub"
    }
}

for col, mapping in mappings.items():
    if col in df_viz.columns:
        df_viz[col] = df_viz[col].map(mapping).astype("category")

In [42]:
df_viz.head()

,Tracking_ID,Warehouse_Zone,Mode_of_Shipment,Customer_Inquiries,Customer_Score,Product_Cost_SAR,Previous_Orders,Product_Priority,Gender,Discount_offered,Product_Weight_g,is_delayed
0,D66B138C54AB,West Hub,Ground Freight,4,1,978.75,2,Medium Priority,Female,9,4158,On Time
1,8FF9538E65E6,Central Port,Standard Sea,3,3,607.50,2,Low Priority,Female,26,1659,Delayed
2,D098284FA376,South Hub,Standard Sea,3,2,828.75,2,Medium Priority,Male,9,4466,On Time
3,9202E9FDFF40,West Hub,Standard Sea,3,5,588.75,4,Medium Priority,Female,2,4640,On Time
4,16B30490A644,North Hub,Premium Air,2,5,1020.00,2,Low Priority,Male,24,3638,Delayed


### Is this a logistics problem?

In [44]:
fig = px.histogram(
    df_viz,
    x="Mode_of_Shipment",
    color="is_delayed",
    barnorm="percent", 
    barmode="stack",
    title="On-time vs Delayed Proportions by Mode of Shipment",
    labels={"is_delayed": "Delivery Status"}
)

fig.update_yaxes(title="Percent of Shipments")
fig.show()


### Is there a regional bottleneck?

In [45]:
fig = px.histogram(
    df_viz,
    x="Warehouse_Zone",
    color="is_delayed",
    barnorm="percent",
    barmode="stack",
    title="On-time vs Delayed Proportions by Warehouse Zone",
    labels={"is_delayed": "Delivery Status", "Warehouse_Zone": "Warehouse Zone"},
)

fig.update_layout(xaxis={"categoryorder": "total descending"})
fig.update_yaxes(title="Percent of Shipments")
fig.show()


### Are heavier products more likely to be delayed?

In [49]:
df_w = df.copy()  

df_w["Weight_Bin"] = pd.cut(
    df_w["Product_Weight_g"],
    bins=[1000, 2000, 3000, 4000, 5000, 6000, 8000],
    include_lowest=True
)

df_w["Weight_Bin"] = df_w["Weight_Bin"].astype(str)

rate = (
    df_w.groupby("Weight_Bin")["is_delayed"]
    .mean()
    .reset_index(name="delay_rate")
)

fig = px.bar(
    rate,
    x="Weight_Bin",
    y="delay_rate",
    text=rate["delay_rate"].mul(100).round(1).astype(str) + "%",
    title="Delay Rate by Product Weight Range"
)

fig.update_yaxes(title="Delay Rate", tickformat=".0%")
fig.update_xaxes(title="Product Weight Range (g)")
fig.show()


### Are discounts being used to compensate for poor logistics?

In [47]:
df_plot = df_viz.copy()
df_plot["Discount_Bin"] = pd.cut(
    df_plot["Discount_offered"],
    bins=[0, 5, 10, 20, 40, 100],
    include_lowest=True,
    right=True,
    labels=["0-5", "6-10", "11-20", "21-40", "41+"]
)

fig = px.histogram(
    df_plot,
    x="Discount_Bin",
    color="is_delayed",
    barnorm="percent",
    barmode="stack",
    title="On-time vs Delayed Proportions by Discount Range",
    labels={"is_delayed": "Delivery Status", "Discount_Bin": "Discount Range"},
)

fig.update_yaxes(title="Percent of Shipments")
fig.show()


### *Summary*

- **Shipping method:** All shipping types are delayed at almost the same rate, so the problem is not how items are shipped.

- **Warehouse location:** All warehouses show similar delays, so the problem is not a specific warehouse.

- **Product weight:** Delays change across weight ranges but without a clear pattern, so weight alone does not explain the issue.

- **Discounts:** Orders with high discounts are almost always delayed, which means discounts are being given after delays happen, not to prevent them.

The main problem is not shipping method, warehouse, or product weight. The company is reacting to delays by giving discounts instead of fixing the cause of the delays, which leads to continued customer dissatisfaction and higher costs.

## Phase 2: Feature Engineering & Baseline
Prepare the data for Machine Learning.

In [52]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

### Drop columns we should not use

In [35]:
df_model = df.drop(columns=["Tracking_ID"])

### Separate Features (X) and Target (y)

In [36]:
X = df_model.drop(columns=["is_delayed"])
y = df_model["is_delayed"]

### Identify Feature Types

In [27]:
categorical_features = [
    "Warehouse_Zone",
    "Mode_of_Shipment",
    "Product_Priority",
    "Gender"
]

numerical_features = [
    "Customer_Inquiries",
    "Customer_Score",
    "Product_Cost_SAR",
    "Previous_Orders",
    "Discount_offered",
    "Product_Weight_g"
]


### Column Transformer (Encoding + Scaling)

In [54]:
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

### Train / Test Split

In [31]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


## Phase 3: Training, Evaluation & Strategy
Build your solution and define your business interventions.

* **Step 3.1:** Train a Classifier.
* **Step 3.2:** Generate a Classification Report and Confusion Matrix.
* **Step 3.3:** **CRITICAL:** List 3 business solution will save our customers based on your model (you can use a 5% Coupon).

In [ ]:
from pycaret.classification import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

### **Step 3.1:** Train a Classifier.

PyCaret requires the full dataset because it manages data splitting, preprocessing, and cross-validation internally to ensure fair and reproducible model comparison.

In [57]:
df_model["is_delayed"] = df_model["is_delayed"].astype(int)

s = setup(
    data=df_model,
    target="is_delayed",
    session_id=123,
    train_size=0.8,
    normalize=True,     
    verbose=False
)

best_model = compare_models(sort="Recall")  
best_model

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.5967,0.0000,1.0000,0.5967,0.7474,0.0000,0.0000,0.0140
dt,Decision Tree Classifier,0.6424,0.0000,0.7036,0.6992,0.7012,0.2559,0.2561,0.0180
ridge,Ridge Classifier,0.6491,0.0000,0.6969,0.7102,0.7032,0.2740,0.2744,0.0160
lda,Linear Discriminant Analysis,0.6478,0.0000,0.6912,0.7111,0.7007,0.2730,0.2735,0.0150
lr,Logistic Regression,0.6421,0.0000,0.6855,0.7063,0.6955,0.2616,0.2621,0.0990
et,Extra Trees Classifier,0.6512,0.0000,0.6731,0.7231,0.6971,0.2873,0.2884,0.0880
svm,SVM - Linear Kernel,0.6450,0.0000,0.6624,0.7247,0.6890,0.2756,0.2805,0.0180
knn,K Neighbors Classifier,0.6217,0.0000,0.6621,0.6908,0.6760,0.2219,0.2222,0.2030
rf,Random Forest Classifier,0.6579,0.0000,0.6340,0.7536,0.6885,0.3153,0.3213,0.0910
lightgbm,Light Gradient Boosting Machine,0.6563,0.0000,0.6221,0.7585,0.6834,0.3156,0.3232,0.2630


DummyClassifier(constant=None, random_state=123, strategy='prior')

In this problem, false negatives are more costly than false positives because missing a delayed shipment directly leads to customer dissatisfaction. Therefore, recall is the most appropriate metric since it focuses on minimizing missed delays. 

Although DummyClassifier achieved perfect recall, it is not a real predictive model. Among valid models, Decision Tree achieved the highest recall and best balance for minimizing missed delayed shipments.

In [ ]:
best_model = create_model("dt")

best_model.get_params() # find df parameters 

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6591,0.0000,0.7429,0.7027,0.7222,0.2819,0.2827
1,0.6420,0.0000,0.6881,0.7049,0.6964,0.2606,0.2607
2,0.6364,0.0000,0.7071,0.6907,0.6988,0.2402,0.2403
3,0.6222,0.0000,0.6976,0.6782,0.6878,0.2096,0.2097
4,0.6364,0.0000,0.6952,0.6952,0.6952,0.2445,0.2445
5,0.6520,0.0000,0.7095,0.7078,0.7087,0.2766,0.2766
6,0.6690,0.0000,0.7286,0.7200,0.7243,0.3104,0.3105
7,0.6307,0.0000,0.6738,0.6970,0.6852,0.2388,0.2390
8,0.6307,0.0000,0.7071,0.6843,0.6956,0.2266,0.2268


{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': 123,
 'splitter': 'best'}

In [ ]:
tuned_dt = tune_model(
    best_model,
    optimize="Recall"
)

tuned_dt.get_params() # fine tuning then find df parameters 

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5966,0.0000,1.0000,0.5966,0.7473,0.0000,0.0000
1,0.5966,0.0000,1.0000,0.5966,0.7473,0.0000,0.0000
2,0.5966,0.0000,1.0000,0.5966,0.7473,0.0000,0.0000
3,0.5966,0.0000,1.0000,0.5966,0.7473,0.0000,0.0000
4,0.5966,0.0000,1.0000,0.5966,0.7473,0.0000,0.0000
5,0.5966,0.0000,1.0000,0.5966,0.7473,0.0000,0.0000
6,0.5966,0.0000,1.0000,0.5966,0.7473,0.0000,0.0000
7,0.5966,0.0000,1.0000,0.5966,0.7473,0.0000,0.0000
8,0.5966,0.0000,1.0000,0.5966,0.7473,0.0000,0.0000


Fitting 10 folds for each of 10 candidates, totalling 100 fits


{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 5,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.2,
 'min_samples_leaf': 5,
 'min_samples_split': 5,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': 123,
 'splitter': 'best'}

Although tuning the Decision Tree for recall achieved a perfect recall score, the model collapsed into predicting all shipments as delayed, resulting in zero discrimination power. Therefore, the tuned model was rejected, and the baseline Decision Tree was selected as it provides a balanced trade-off between recall and precision.

In [64]:
pipe = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", DecisionTreeClassifier(random_state=123))
])

param_grid = {
    "model__max_depth": [3, 5, 8, 12, 20],
    "model__min_samples_split": [5, 10, 20],
    "model__min_samples_leaf": [2, 4, 8],
    "model__criterion": ["gini", "entropy"],
    "model__max_features": [None, "sqrt", "log2"]
}

cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring="recall",
    cv=cv_strategy,
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)

print("Best Params:", grid.best_params_)
print("Best CV Recall:", grid.best_score_)
best_model = grid.best_estimator_


Fitting 5 folds for each of 270 candidates, totalling 1350 fits
Best Params: {'model__criterion': 'gini', 'model__max_depth': 3, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 8, 'model__min_samples_split': 5}
Best CV Recall: 0.8861904761904761


Grid Search identified an optimized Decision Tree configuration that maximizes recall while maintaining model simplicity.

### **Step 3.2:** Generate a Classification Report and Confusion Matrix.

In [73]:

best_pipe = grid.best_estimator_

y_pred = best_pipe.predict(X_test)

print("Best Params:", grid.best_params_)
print("Best CV Recall:", grid.best_score_)

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Best Params: {'model__criterion': 'gini', 'model__max_depth': 3, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 8, 'model__min_samples_split': 5}
Best CV Recall: 0.8861904761904761

Confusion Matrix:
[[165 545]
 [125 925]]

Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.23      0.33       710
           1       0.63      0.88      0.73      1050

    accuracy                           0.62      1760
   macro avg       0.60      0.56      0.53      1760
weighted avg       0.60      0.62      0.57      1760



In [78]:
cm = confusion_matrix(y_test, y_pred)

cm_norm = cm / cm.sum(axis=1, keepdims=True)
labels = ["Not Delayed", "Delayed"]  

fig_cm_norm = px.imshow(
    cm_norm,
    text_auto=".2%",
    x=labels,
    y=labels,
    color_continuous_scale="Blues",
    labels={"x": "Predicted", "y": "Actual", "color": "Percent"},
    title="Interactive Confusion Matrix (Normalized)"
)
fig_cm_norm.update_layout(height=450, width=650)
fig_cm_norm.show()


### **Step 3.3:** **CRITICAL:** List 3 business solution will save our customers based on your model (you can use a 5% Coupon).

### 📝 Business Strategy Recommendations
1. Use the model to identify shipments likely to be delayed early and notify customers in advance with a 5% coupon to reduce dissatisfaction.
2. Prioritize shipments predicted as high delay risk in warehouse and delivery operations to prevent delays before they happen.
3. Target discounts and customer support only to high-risk delayed shipments to reduce costs while improving customer retention.

## Phase 4: The Business Value Calculator
Calculate the SAR impact of your model using the following values:
* **+ SAR 75.00**: True Positive (Saved Customer)
* **- SAR 18.75**: False Positive (5% Wasted Coupon)
* **- SAR 187.50**: False Negative (Lost Lifetime Value)

In [81]:
TP = cm[1, 1]  
FP = cm[0, 1]  
FN = cm[1, 0] 

VALUE_TP = 75.00     
COST_FP = -18.75    
COST_FN = -187.50    

total_value = (
    TP * VALUE_TP +
    FP * COST_FP +
    FN * COST_FN
)

print(f"Total Business Value Generated by the Model: {total_value:.2f} SAR")

Total Business Value Generated by the Model: 35718.75 SAR


This calculation translates the model’s predictions into real business value by rewarding saved customers and penalizing wasted coupons and missed delayed shipments in SAR.